# Reality Stone - RS-ULF Finetuning on Colab

Riemannian-Semantic Unified Latent Flow (RS-ULF) 기반 LLM 파인튜닝

- RS-ULF: Transformer의 Attention/FFN을 리만 기하학 기반으로 압축
- Rust 코어 연산으로 빠른 추론
- 메모리 효율적인 파인튜닝


In [ ]:
# 1. Install reality_stone
%pip install reality_stone transformers datasets accelerate -q


In [ ]:
# 2. Check installation
import reality_stone as rs
print(f"Version: {rs.__version__}")
print(f"Rust extension: {rs._has_rust_ext}")
print(f"CUDA available: {rs._has_cuda}")
print(f"RSULFLayer available: {rs.RSULFLayer is not None}")


In [ ]:
# 3. Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")


## 4. Load Base Model


In [ ]:
MODEL_NAME = "beomi/Llama-3-Open-Ko-8B"
TARGET_RANK = 256

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)
print(f"Model loaded: {MODEL_NAME}")


## 5. Extract Weights and Create RS-ULF Layers


In [ ]:
from reality_stone.models.transformer_converter import (
    extract_transformer_layer_weights,
    verify_fold_consistency,
)

def create_rsulf_layer(model, layer_idx, target_rank=256):
    weights = extract_transformer_layer_weights(model, layer_idx)
    
    WQ = weights['WQ'].float()
    WK = weights['WK'].float()
    W1 = weights['W1'].float()
    W2 = weights['W2'].float()
    
    consistency = verify_fold_consistency(WQ, WK, target_rank)
    print(f"Layer {layer_idx}: {consistency}")
    
    WK_np = WK.numpy()
    if WK.shape[0] < WQ.shape[0]:
        WK_np = np.tile(WK_np, (WQ.shape[0] // WK.shape[0], 1))
    
    rsulf = rs.RSULFLayer(
        wq=WQ.numpy(),
        wk=WK_np,
        w1=W1.numpy(),
        w2=W2.numpy(),
        d_model=WQ.shape[1],
        r=target_rank,
    )
    
    return rsulf, consistency

n_layers = 4
rsulf_layers = []
for i in range(n_layers):
    layer, consistency = create_rsulf_layer(model, i, TARGET_RANK)
    rsulf_layers.append(layer)
    
print(f"\nCreated {len(rsulf_layers)} RS-ULF layers")


## 6. RS-ULF PyTorch Wrapper


In [ ]:
class RSULFWrapper(nn.Module):
    def __init__(self, rsulf_layer):
        super().__init__()
        self.rsulf = rsulf_layer
        self.v_mem = None
        
    def forward(self, x):
        batch, seq, dim = x.shape
        x_np = x.detach().cpu().float().numpy().reshape(-1, dim)
        
        v_mem_np = self.v_mem.numpy() if self.v_mem is not None else None
        out_np, v_new = self.rsulf.forward(x_np, v_mem_np)
        
        self.v_mem = torch.from_numpy(v_new)
        return torch.from_numpy(out_np).view(batch, seq, -1).to(x.device, x.dtype)

class RSULFLMHead(nn.Module):
    def __init__(self, rsulf_layers, hidden_size, vocab_size):
        super().__init__()
        self.rsulf_wrappers = nn.ModuleList([RSULFWrapper(l) for l in rsulf_layers])
        self.proj = nn.Linear(rsulf_layers[0].r, hidden_size)
        self.lm_head = nn.Linear(hidden_size, vocab_size, bias=False)
        
    def forward(self, hidden_states):
        x = hidden_states
        for wrapper in self.rsulf_wrappers:
            x = wrapper(x)
        x = self.proj(x)
        return self.lm_head(x)

hidden_size = model.config.hidden_size
vocab_size = model.config.vocab_size

rsulf_head = RSULFLMHead(rsulf_layers, hidden_size, vocab_size).to(device)
print(f"RS-ULF LM Head created: hidden={hidden_size}, vocab={vocab_size}")


## 7. Load Dataset


In [ ]:
dataset = load_dataset("heegyu/kowikitext", split="train[:10000]")

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt",
    )

tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=dataset.column_names)
tokenized.set_format("torch")

dataloader = DataLoader(tokenized, batch_size=4, shuffle=True)
print(f"Dataset loaded: {len(tokenized)} samples")


## 8. Training Loop


In [ ]:
for param in model.parameters():
    param.requires_grad = False

optimizer = torch.optim.AdamW([
    {"params": rsulf_head.proj.parameters(), "lr": 1e-4},
    {"params": rsulf_head.lm_head.parameters(), "lr": 5e-5},
])

num_epochs = 3
model.eval()

for epoch in range(num_epochs):
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for batch in pbar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        
        with torch.no_grad():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True,
            )
            hidden_states = outputs.hidden_states[-1]
        
        logits = rsulf_head(hidden_states.float())
        
        shift_logits = logits[:, :-1, :].contiguous()
        shift_labels = input_ids[:, 1:].contiguous()
        
        loss = F.cross_entropy(
            shift_logits.view(-1, vocab_size),
            shift_labels.view(-1),
            ignore_index=tokenizer.pad_token_id,
        )
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} - Average Loss: {avg_loss:.4f}")


## 9. Save and Test


In [ ]:
save_path = "rsulf_lm_head.pt"
torch.save({
    "proj": rsulf_head.proj.state_dict(),
    "lm_head": rsulf_head.lm_head.state_dict(),
    "rsulf_components": [l.export_components() for l in rsulf_layers],
}, save_path)
print(f"Model saved to {save_path}")


In [ ]:
@torch.no_grad()
def generate(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    outputs = model(**inputs, output_hidden_states=True)
    hidden = outputs.hidden_states[-1]
    
    generated = inputs.input_ids.tolist()[0]
    
    for _ in range(max_new_tokens):
        logits = rsulf_head(hidden.float())
        next_token = logits[:, -1, :].argmax(dim=-1).item()
        
        if next_token == tokenizer.eos_token_id:
            break
            
        generated.append(next_token)
        
        new_input = torch.tensor([[next_token]], device=device)
        outputs = model(input_ids=new_input, output_hidden_states=True)
        hidden = outputs.hidden_states[-1]
    
    return tokenizer.decode(generated, skip_special_tokens=True)

test_prompts = [
    "인공지능의 미래는",
    "한국의 역사에서",
]

for prompt in test_prompts:
    print(f"Prompt: {prompt}")
    print(f"Generated: {generate(prompt)}")
    print("-" * 50)
